# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [60]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [61]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [62]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [63]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [64]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [65]:
# Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION ={'class':'SimpleStrategy',\
                   'replication_factor':1}""")

#### Set Keyspace

In [66]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

>*1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4.*

>*2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.*

>*3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.*




### *Query 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4*


In [67]:
#
# As design of Cassandra tables are mainly based on the type and nature of the query, to implement the 
# Query 1 (as above) we can think of (session_id, item_in_session) as primary key
# (partition key=session_id,clustering key=item_in_session).In Sparkify App, each time a user logged 
# in is assigned a unique session id. In a particular session, a user can listen many songs, it also assign   
# a unique number for each song in a given session. Hence, to get unique record in this scenario 
# (session_id, item_in_session) would be sufficient.
# Other than this we need to include columns such as: artist, song_title and song_length in the table design.
#
query="CREATE TABLE IF NOT EXISTS music_app_history"
query=query+"(session_id INT,item_in_session INT,artist TEXT,song_title TEXT,song_length FLOAT,\
              PRIMARY KEY(session_id,item_in_session))"
session.execute(query);                    

In [68]:
# Reading data from csv file to load into Cassandra tables.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history(session_id,item_in_session,artist,song_title,song_length) "
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

In [69]:
# Adding the SELECT statement to verify the data was entered into the table.
#
# We are to find out song's details for the session_id=338 and item_in_session=4
# so clealy our WHERE clause will conatin session_id and item_in_session 
# Song's details we want are artist name, song name and song lengh so
# SELECT clause will contain these columns.
#
query=("SELECT artist, song_title, song_length FROM music_app_history WHERE" \
                      " session_id = 338 and item_in_session = 4;")
rows=session.execute(query)
dict={}
df=pd.DataFrame(dict,columns=['artist','song_title','song_length'])
for row in rows:
    #print(row.firstname,row.lastname)
    dict={'artist':row.artist,'song_title':row.song_title,'song_length':row.song_length}
    #print(dict)
    df=df.append(dict,ignore_index=True)    
print("Query 1.:{}\n".format(query))    
print("OUTPUT:\n")
df    

Query 1.:SELECT artist, song_title, song_length FROM music_app_history WHERE session_id = 338 and item_in_session = 4;

OUTPUT:



,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### *Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

In [70]:
#
# In this query we filter data based on user_id and session-id, so these two columns will be part of
# partition key and as the songs are required to be sorted by item_in_session so it will be included in
# clustering key. Hence,our primary key would be ((user_id,session_id),item_in_session)).
# As previously we have seen in Query 1 that even (session_id,item_in _session) pair is able to 
# identify the unique row. In Cassandra query dictates the DB design, so our PK should be according to that only.
# In addition to above keys columns we need to include artist name, song_title, first name and lastname
# in our table design.
#
query="CREATE TABLE IF NOT EXISTS music_app_session"
query=query+"(user_id int,session_id INT,item_in_session INT,artist TEXT,song_title TEXT,firstname TEXT,lastname TEXT,\
              PRIMARY KEY((user_id,session_id),item_in_session)) WITH CLUSTERING ORDER BY (item_in_session DESC);"
session.execute(query); 

In [71]:
# Reading data from csv file to load into Cassandra tables.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_session(user_id,session_id,item_in_session,artist,song_title,firstname,lastname)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [72]:
# Adding the SELECT statement to verify the data was entered into the table.
#
# Here in this query data needs to be filtered on the basis of user_id and session_id
# so WHERE contains these two columns and in the SELECT statement we wants columns such as
# artist name, song name, number of items in session, firstname of the user, last name of the user.
#

query=("SELECT artist, song_title, item_in_session, firstname, lastname FROM music_app_session WHERE " \
        "user_id = 10 and session_id = 182;")
rows=session.execute(query)
dict={}
df=pd.DataFrame(dict)
for row in rows:
    #print(row.firstname,row.lastname)
    dict={'artist':row.artist,'song_title':row.song_title,'item_in_session':row.item_in_session,\
          'FirtName':row.firstname,'LastName':row.lastname}
    #print(dict)
    df=df.append(dict,ignore_index=True)                     
print("Query 2.:{}\n".format(query))    
print("OUTPUT:\n")
df 

Query 2.:SELECT artist, song_title, item_in_session, firstname, lastname FROM music_app_session WHERE user_id = 10 and session_id = 182;

OUTPUT:



,FirtName,LastName,artist,item_in_session,song_title
0,Sylvie,Cruz,Lonnie Gordon,3.0,Catch You Baby (Steve Pitron & Max Sanna Radio...
1,Sylvie,Cruz,Sebastien Tellier,2.0,Kilometer
2,Sylvie,Cruz,Three Drives,1.0,Greece 2000
3,Sylvie,Cruz,Down To The Bone,0.0,Keep On Keepin' On


### *Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.*

In [73]:
#  
# In this query, data is filtered based on song names so 'song_title' will be our partion key.
# As we can see, 'song_title' alone can not uniquely define row so we need to think of other
# column as clustering key to form the primary key. Also, we are looking for minimum number of 
# columns in as a part of clustering key, the 'user_id' would be a good choice.
# So our primary key would be (song_title,user_id).
# Other than this we need to include firstname and lastname in our table design.
#
query="CREATE TABLE IF NOT EXISTS music_app_song"
query=query+"(song_title TEXT,user_id INT,firstname TEXT,lastname TEXT,\
              PRIMARY KEY(song_title,user_id))"
session.execute(query);                  

In [74]:
# Reading data from csv file to load into Cassandra tables.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
# Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_song(song_title,user_id,firstname,lastname) "
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [75]:
# Adding the SELECT statement to verify the data was entered into the table.
#
# In the query, data is filtered based on song name hence where clause contains 'song_title'
# In addition to that, in SELECt clasuse we incluse firstname of the user and lastname of the user.
#
query=("SELECT firstname, lastname FROM music_app_song "\
       "WHERE song_title='All Hands Against His Own';")
rows=session.execute(query)
dict={}
df=pd.DataFrame(dict,columns=['FirstName','LastName'])
for row in rows:
    #print(row.firstname,row.lastname)
    dict={'FirstName':row.firstname,'LastName':row.lastname}
    #print(dict)
    df=df.append(dict,ignore_index=True)                     
print("Query 2.:{}\n".format(query))    
print("OUTPUT:\n")
df                              

Query 2.:SELECT firstname, lastname FROM music_app_song WHERE song_title='All Hands Against His Own';

OUTPUT:



,FirstName,LastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### *Drop the tables before closing out the sessions*

In [76]:
# Dropping the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS music_app_history;");
session.execute("DROP TABLE IF EXISTS music_app_session;");
session.execute("DROP TABLE IF EXISTS music_app_song;");

### *Close the session and cluster connection*

In [77]:
session.shutdown()
cluster.shutdown()